# Classifying WBCD using Decision Tree (C4.5)

In [36]:
!pip install chefboost

import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

df = pd.read_csv("wbcd.csv")
df = df.rename(columns={"Class":"Decision"})
df

,ClumpThickness,UniformityOfCellSize,UniformityofCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,Decision
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2,1,1,1,2
695,2,1,1,1,2,1,1,1,1,2
696,5,10,10,3,7,3,8,10,2,4
697,4,8,6,4,3,4,10,6,1,4


> Accuracy

In [37]:
def accuracy(y_t, y_p):
    return (np.sum(y_t == y_p)/ len(y_t))

### Method - 1 : Removing missing data
> Removing missing data

In [38]:
df_non_empty = df.copy(deep=True)
df_non_empty = df_non_empty[df_non_empty['BareNuclei'] != '?'].astype({'BareNuclei': 'int64'})
df_non_empty

,ClumpThickness,UniformityOfCellSize,UniformityofCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,Decision
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2,1,1,1,2
695,2,1,1,1,2,1,1,1,1,2
696,5,10,10,3,7,3,8,10,2,4
697,4,8,6,4,3,4,10,6,1,4


> Splitting into training and testing sets

In [39]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_non_empty, test_size=0.3)
df_train

,ClumpThickness,UniformityOfCellSize,UniformityofCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,Decision
471,6,1,1,3,2,1,1,1,1,2
444,5,1,1,6,3,1,2,1,1,2
497,4,2,1,1,2,1,1,1,1,2
94,2,1,1,1,2,1,3,1,1,2
277,1,1,1,1,2,1,2,1,1,2
...,...,...,...,...,...,...,...,...,...,...
403,3,1,1,1,2,4,1,1,1,2
131,2,1,1,1,2,1,3,1,1,2
238,8,10,10,8,6,9,3,10,10,4
446,1,1,1,1,2,1,1,1,1,2


> Training decision tree classifier model

In [40]:
from chefboost import Chefboost as chef

model = chef.fit(df_train, {'algorithm': 'C4.5'})

x_test = df_test.drop(['Decision'],axis=1)
y_test = df_test[['Decision']]

x_test_array = x_test.to_numpy().tolist()

y_predicted = []
for i in x_test_array:
    y_predicted.append(chef.predict(model, i))

That's why, the algorithm is set to Regression to handle the data set.
Regression  tree is going to be built...


/usr/local/lib/python3.7/dist-packages/chefboost/training/Preprocess.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = np.where(df[column_name] <= winner_threshold, "<="+str(winner_threshold), ">"+str(winner_threshold))
/usr/local/lib/python3.7/dist-packages/chefboost/training/Preprocess.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = np.where(df[column_name] <= winner_threshold, "<="+str(winner_threshold), ">"+str(winner_threshold))
/usr/local/lib/python3.7/dis

-------------------------
finished in  3.239072799682617  seconds
-------------------------
Evaluate  train set
-------------------------
MAE:  0.02092050209205021
MSE:  0.02092050209205021
RMSE:  0.14463921353509293
RAE:  0.05076730825668096
RRSE:  0.15233412578445724
Mean:  2.6861924686192467
MAE / Mean:  0.778816199376947 %
RMSE / Mean:  5.384543930667791 %


> Accuracy

In [41]:
import numpy as np

y_true = np.asarray(y_test.values.T.tolist()[0])
y_pred = np.asarray(y_predicted)

accuracy(y_true, y_pred)

0.9463414634146341

### Method - 2 : Replacing missing data with 1

> Replacing

In [42]:
df_replaced = df.copy(deep=True)
df_replaced['BareNuclei']  = df_replaced['BareNuclei'].replace({'?': 1})
df_replaced = df_replaced.astype({'BareNuclei': 'int64'})
df_replaced

,ClumpThickness,UniformityOfCellSize,UniformityofCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,Decision
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2,1,1,1,2
695,2,1,1,1,2,1,1,1,1,2
696,5,10,10,3,7,3,8,10,2,4
697,4,8,6,4,3,4,10,6,1,4


> Splitting second set

In [43]:
df_train_1, df_test_1 = train_test_split(df_replaced, test_size=0.3)
df_train_1

,ClumpThickness,UniformityOfCellSize,UniformityofCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,Decision
129,1,1,1,1,10,1,1,1,1,2
246,8,10,10,8,5,10,7,8,1,4
279,10,5,7,3,3,7,3,3,8,4
697,4,8,6,4,3,4,10,6,1,4
672,2,1,1,1,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
629,4,1,1,1,2,1,1,1,1,2
694,3,1,1,1,3,2,1,1,1,2
113,10,10,10,3,10,8,8,1,1,4
309,3,1,1,1,2,5,5,1,1,2


> Training

In [44]:
model_1 = chef.fit(df_train_1, {'algorithm': 'C4.5'})

x_test_1 = df_test_1.drop(['Decision'],axis=1)
y_test_1 = df_test_1[['Decision']]

x_test_array_1 = x_test_1.to_numpy().tolist()

y_predicted_1 = []
for i in x_test_array_1:
    y_predicted_1.append(chef.predict(model_1, i))

That's why, the algorithm is set to Regression to handle the data set.
Regression  tree is going to be built...


/usr/local/lib/python3.7/dist-packages/chefboost/training/Preprocess.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = np.where(df[column_name] <= winner_threshold, "<="+str(winner_threshold), ">"+str(winner_threshold))
/usr/local/lib/python3.7/dist-packages/chefboost/training/Preprocess.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = np.where(df[column_name] <= winner_threshold, "<="+str(winner_threshold), ">"+str(winner_threshold))
/usr/local/lib/python3.7/dis

-------------------------
finished in  3.183739423751831  seconds
-------------------------
Evaluate  train set
-------------------------
MAE:  0.03817314246762099
MSE:  0.06680299931833673
RMSE:  0.25846276195679857
RAE:  0.09014453108339651
RRSE:  0.2706343946552083
Mean:  2.703476482617587
MAE / Mean:  1.4120020171457386 %
RMSE / Mean:  9.560385067842246 %


> Accuracy

In [45]:
y_true_1 = np.asarray(y_test_1.values.T.tolist()[0])
y_pred_1 = np.asarray(y_predicted_1)

accuracy(y_true_1, y_pred_1)

0.9476190476190476